<a href="https://colab.research.google.com/github/ryuchallenger/red_project/blob/main/SCA_PC_AL_MC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
### library

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [96]:
### Data

PC=pd.read_csv("/content/drive/MyDrive/SCA/SCA21_NON47/SCA_rawdata/PC.tsv", sep="\t", index_col="TRID")
NC=pd.read_csv("/content/drive/MyDrive/SCA/SCA21_NON47/SCA_rawdata/NC.tsv", sep="\t")
STATUS=pd.read_csv("/content/drive/MyDrive/SCA/SCA21_NON47/SCA21/PC_AL_all_status_long.tsv", sep="\t")
STRchive=pd.read_csv("/content/drive/MyDrive/SCA/SCA21_NON47/NON47/Genotyping-STRchive-disease-loci-v2.4.2.hg38.TRGT.B2FLLAlV.bed", sep="\t", header=None)
STRchive.columns=["chromosome", "start", "end", "description"]

In [4]:
### SCA_RED

SCA_RED = ["DRPLA_ATN1",
           "SCA1_ATXN1",
           "SCA2_ATXN2",
           "SCA3_ATXN3",
           "SCA4_ZFHX3",
           "SCA6_CACNA1A",
           "SCA7_ATXN7",
           "SCA8_ATXN8OS",
           "SCA10_ATXN10",
           "SCA12_PPP2R2B", #less commonly
           "SCA17_TBP",
           "SCA27B_FGF14",
           "FRDA_FXN",
           "CANVAS_RFC1",
           "SCA31_BEAN1", #less commonly
           "FXS_FMR1", #less commonly
           "SCA36_NOP56", #less commonly
           "SCA37_DAB1"]

OMIM = [
    "The ATN1 gene encodes atrophin-1, a member of a class of evolutionarily conserved transcriptional corepressors involved in nuclear signaling. ATN1 is believed to play a role as a nuclear transcriptional regulator important for brain and other organ system development (summary by Palmer et al., 2019).",
    "ATXN1 binds RNA, associates with large protein complexes, and interacts with a vast network of proteins. ATXN1 is thought to be involved in transcriptional repression and to regulate Notch (see 190198)- and Capicua (CIC; 612082)-controlled developmental processes (summary by Bergeron et al., 2013).",
    "Ataxin-2, the protein encoded by the ATXN2 gene, contains a polyglutamine tract, long expansions (greater than 33 repeats) of which result in spinocerebellar ataxia-2 (SCA2; 183090), an autosomal dominant form of olivopontocerebellar atrophy. Intermediate-length expansions (27-33 glutamines) contribute to susceptibility to amyotrophic lateral sclerosis (ALS13; see 183090).",
    "ATXN3 has deubiquitinase activity and appears to be a component of the ubiquitin proteasome system. It may also have roles in transcriptional regulation and neuroprotection (summary by Haacke et al., 2006).",
    "ZFHX3 is a transcriptional regulator that interacts with AT motifs in target genes. ZFHX3 is involved in regulation of circadian rhythm (Parsons et al., 2015) and appetite and body weight (Nolan et al., 2023).",
    "The CACNA1A gene encodes the transmembrane pore-forming subunit of the P/Q-type or CaV2.1 voltage-gated calcium channel (VGCC) (Kordasiewicz et al., 2006). Voltage-dependent Ca(2+) channels not only mediate the entry of Ca(2+) ions into excitable cells but are also involved in a variety of Ca(2+)-dependent processes, including muscle contraction, hormone or neurotransmitter release, and gene expression. Diriong et al. (1995) noted that calcium channels are multisubunit complexes and that the channel activity is directed by a pore-forming alpha-1 subunit, which is often sufficient to generate voltage-sensitive Ca(2+) channel activity. There are at least 6 classes of alpha-1 subunits: alpha-1A, B, C, D, E, and S, which are derived from 6 genes representing members of a gene family. The auxiliary subunits beta (e.g., 114207), alpha-2/delta, and gamma (e.g., 114209) regulate channel activity. In addition to full-length CACNA1A, use of an internal ribosomal entry site in the CACNA1A transcript generates the CACNA1A C-terminal polypeptide, or alpha-1ACT, which functions as a transcription factor that mediates cerebellar development (Du et al., 2013).",
    "ATXN7 is a transcription factor that appears to be critically important for chromatin remodeling at the level of histone acetylation and deubiquitination. It is a core component of 2 different transcription coactivator complexes: the SPT3 (SUPT3H; 602947)/TAF9 (600822)/GCN5 (KAT2A; 602301) acetyltransferase (STAGA) complex, which has histone acetyltransferase activity, and the USP22 (612116) deubiquitination complex (summary by Sopher et al., 2011).",
    "Spinocerebellar ataxia-8 (SCA8; 608768) is a neurodegenerative disorder caused by a CTG/CAG trinucleotide repeat expansion on chromosome 13q21 (see 603680.0001 and 613289.0001). Two genes span the CTG/CAG repeat and are expressed in opposite directions: ATXN8 (613289), which encodes a nearly pure polyglutamine expansion protein in the CAG direction, and ATXN8OS, which, when transcribed, produces a noncoding CUG expansion RNA (Moseley et al., 2006).",
    "By positional cloning in the spinocerebellar ataxia-10 (SCA10; 603516) candidate region on chromosome 22q13-qter, Matsuura et al. (2000) identified a gene, ataxin-10 (ATXN10), encoding a deduced 475-amino acid protein with 82% identity with a presumed mouse ortholog (E46). By Northern blot analysis, Wakamiya et al. (2006) identified a 2-kb mRNA ATXN10 transcript.",
    "The PPP2R2B gene encodes a brain-specific regulatory subunit B of protein phosphatase 2. Protein phosphatase 2A (PP2A), a heterotrimeric serine/threonine phosphatase, has been implicated in a variety of regulatory processes, including cell growth and division, muscle contraction, and gene transcription. PP2A is composed of a 36-kD catalytic subunit (176915), a highly homologous 65-kD structural subunit (176915), and any of several different regulatory subunits that control its specificity, including PPP2R2B (Mayer et al., 1991).",
    "The RNA polymerase II transcription factor D (TFIID; see 313650) is a multisubunit complex essential for the expression of most, if not all, protein-encoding genes. The DNA-binding subunit of TFIID is the TATA box-binding protein (TBP).",
    "The FGF14 gene encodes fibroblast growth factor-14, which is highly expressed in the brain, particularly in Purkinje cells where it interacts with voltage-gated channels to regulate neuronal excitability. FGF14 also plays a role in synaptic plasticity and neurogenesis in the hippocampus (summary by Piarroux et al., 2020). FGF14 belongs to a subclass of fibroblast growth factors that are expressed in the developing and adult central nervous system (Smallwood et al., 1996). For a discussion on the FHF gene family, see FGF12 (FHF1; 601513).",
    "Frataxin is a nuclear-encoded mitochondrial iron chaperone involved in iron-sulfur biogenesis and heme biosynthesis. Some studies have also suggested that frataxin functions as an iron storage molecule, an antioxidant, and a tumor suppressor (summary by Schmucker et al. (2008)).",
    "The RFC1 gene encodes the large subunit of replication factor C, a 5-subunit DNA polymerase accessory protein required for the coordinated synthesis of both DNA strands during replication or after DNA damage (summary by Cortese et al., 2019). It is a DNA-dependent ATPase that binds in a structure-specific manner to the 3-prime end of a primer hybridized to a template DNA, an activity thought intrinsic to the 140-kD component of this multisubunit complex (Bunz et al., 1993).",
    "By Southern blot analysis of a 900-kb critical region on chromosome 16q21-q22, followed by sequencing analysis, Sato et al. (2009) identified 2.5- to 3.8-kb insertions containing pentanucleotide repeats including a (TGGAA)n sequence in all 160 affected individuals from 98 families with SCA31 (117210). PCR amplification followed by sequencing showed that the insertion consisted of a preceding TCAC sequence, and 3 pentanucleotide repeat components (TGGAA)n, (TAGAA)n, and (TAAAA)n in all patients tested. In a homozygous patient from whom the 900-kb critical region was derived, the authors found a (TGGAA)n sequence of greater than 110 repeats and a (TAAAATAGAA)n sequence of greater 112 repeats, both of which were too long to be read through. The (TGGAA)n and (TAAAATAGAA)n sequences were separated by a bridging sequence and (TAGAA)46. The insertions were located in introns of the BEAN (612051) and TK2 (188250) genes, which are on opposite strands and transcribed in opposite directions. These insertions were not identified in 99.77% of 800 Japanese and 60 American chromosomes, or in individuals with SCA4 (600223). However, 2 (0.23%) of 860 control chromosomes did carry similar smaller 1.5- or 2.0-kb insertions without (TGGAA)n sequences. Sato et al. (2009) concluded that the insertions in SCA31 patients exerted their toxicity either because of their length or the (TGGAA)n sequence, or because of both. The length of the SCA31 insertion was inversely correlated with the age at disease onset. Further analysis showed that the insertion site was identical for all insertions and was located at an Alu sequence. A single-nucleotide change in an intron of the TK2 gene segregated with SCA31 but was not considered to be pathogenic. The repeat insertions did not appear to cause splicing abnormalities or alterations in the expression levels of BEAN, TK2, or other nearby genes. Sato et al. (2009) demonstrated that the insertion transcribed in the direction of BEAN formed RNA foci in approximately 30 to 50% of Purkinje cell nuclei from SCA31 patients but not in cells from controls. RNA foci were not observed for antisense probes corresponding to TK2 transcripts in SCA31 or control brains. The splicing factors SFRS1 (600812) and SFRS9 (601943) were found to directly bind to (UGGAA)n, the transcribed sequence of (TGGAA)n, in vitro. In silico analysis showed that (TGGAA)n was abundant in centromeres of several human chromosomes, suggesting a role in heterochromatin or chromosomal structure.",
    "The selective RNA-binding protein FMRP forms a messenger ribonucleoprotein complex that associates with polyribosomes, suggesting that it is involved in translation (Jin et al., 2004).",
    "NOP56 is a component of the box C/D small nucleolar ribonucleoprotein complexes that direct 2-prime-O-methylation of pre-ribosomal RNA (rRNA) during its maturation. NOP56 is predicted to function in an early to middle step in pre-rRNA processing (Hayano et al., 2003).",
    "The reelin (RELN; 600514) signaling pathway plays a critical role in the correct positioning of neurons within the developing brain. Animal studies have shown that DAB1 serves as an intracellular adaptor that is tyrosine phosphorylated when reelin binds to the lipoprotein receptors VLDLR (192977) and APOER2 (LRP8; 602600) on the surface of neurons (Huang et al., 2005)."
]


result =(
    df[
    df["TRID"].isin(SCA_RED) &
    df["status"].isin(["PATHOGENIC", "INTERMEDIATE"])
    ]
    .set_index("sample")
    .sort_index()
)

exclude_list = ["SCA31_BEAN1"]
result = result[~result["TRID"].isin(exclude_list)]


In [5]:
### sample_total

sample_list=(
    PC
    .filter(like=":AL")
    .columns
    .str.replace(':AL', '', regex=False)
    )

motifs=(
    PC
    .loc[SCA_RED, 'MOTIFS']
    .str.split(",", expand=True)
)

motifs.columns=[f"motif_{i+1}" for i in range(motifs.shape[1])]

sample_AL={}
sample_MC={}
sample_total={}
for sample in sample_list:
    sample_AL[f"{sample}_AL"]=PC.loc[SCA_RED, f'{sample}:AL'].str.split(",", expand=True)
    sample_AL[f"{sample}_AL"].columns=[f"{sample}_AL_allele_{i+1}" for i in range(sample_AL[f"{sample}_AL"].shape[1])]

    sample_MC[f"{sample}_MC"]=PC.loc[SCA_RED, f'{sample}:MC'].str.split(",", expand=True)
    sample_MC[f"{sample}_MC"].columns=[f"{sample}_MC_allele_{i+1}" for i in range(sample_MC[f"{sample}_MC"].shape[1])]
    sample_MC_1=sample_MC[f"{sample}_MC"].iloc[:, 0].str.split("_", expand=True)
    sample_MC_1.columns=[f"{sample}_MC_allele_1_motif_{i+1}" for i in range(sample_MC_1.shape[1])]
    sample_MC_2=sample_MC[f"{sample}_MC"].iloc[:, 1].str.split("_", expand=True)
    sample_MC_2.columns=[f"{sample}_MC_allele_2_motif_{i+1}" for i in range(sample_MC_2.shape[1])]

    df_list=[motifs, sample_AL[f"{sample}_AL"], sample_MC_1, sample_MC_2]
    sample_total[f"{sample}"]=pd.concat(df_list, keys=["motifs", "AL", "MC_allele_1", "MC_allele_2"], axis=1)
    sample_total[f"{sample}"].to_excel(f"{sample}.xlsx")

pd.concat(sample_total, axis=1).to_excel("PC_total.xlsx")

In [ ]:
### AL scatter plot
sample_total_key_list=list(sample_total.keys())

fig, axes = plt.subplots(len(sample_total_key_list), 2, figsize=(15,60))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

def scatter_MC(ax, data, column, title):
  data.plot.scatter(ax=ax, rot=45,
                    x='TRID', y=column,
                    xlabel='TRID', ylabel='AL',
                    ylim=(0,1000),
                    title=title)

for i, key in enumerate(sample_total_key_list):
  df = sample_total[key].AL.fillna(0).astype(int)
  df.reset_index(inplace=True)

  scatter_MC(axes[i*2], df, df.columns[1], f'{key}_allele_1')
  scatter_MC(axes[i*2+1], df, df.columns[2], f'{key}_allele_2')

plt.tight_layout()
plt.show()

In [94]:
### practice

sample

,TRID,sample,allele,TR,status,status_fmt
0,ADTKD_MUC1,B02_35311,allele1,2621.0,PATHOGENIC,pathogenic(2621.0)
1,ADTKD_MUC1,B02_35311,allele2,2621.0,PATHOGENIC,pathogenic(2621.0)
2,ADTKD_MUC1,B10_31534,allele1,2621.0,PATHOGENIC,pathogenic(2621)
3,ADTKD_MUC1,B10_31534,allele2,2621.0,PATHOGENIC,pathogenic(2621)
4,ADTKD_MUC1,B11_33267,allele1,2621.0,PATHOGENIC,pathogenic(2621)
...,...,...,...,...,...,...
3061,pre-MIR7-2_CHNG3,SCA_15277,allele2,19.0,PATHOGENIC,pathogenic(19)
3062,pre-MIR7-2_CHNG3,SCA_49841,allele1,19.0,PATHOGENIC,pathogenic(19)
3063,pre-MIR7-2_CHNG3,SCA_49841,allele2,19.0,PATHOGENIC,pathogenic(19)
3064,pre-MIR7-2_CHNG3,SCA_52664,allele1,19.0,PATHOGENIC,pathogenic(19.0)
